In [16]:
import cv2
import numpy as np
import sys


LABELS = ['person','bicycle','car','motorbike','aeroplane','bus','train','truck','boat','traffic light',
          'fire hydrant','stop sign','parking meter','bench','bird','cat','dog','horse','sheep','cow',
          'elephant','bear','zebra','giraffe','backpack','umbrella','handbag','tie','suitcase','frisbee',
          'skis','snowboard','sports ball','kite','baseball bat','baseball glove','skateboard','surfboard',
          'tennis racket','bottle','wine glass','cup','fork','knife','spoon','bowl','banana','apple','sandwich',
          'orange','broccoli','carrot','hot dog','pizza','donut','cake','chair','sofa','pottedplant','bed',
          'diningtable','toilet','tvmonitor','laptop','mouse','remote','keyboard','cell phone','microwave','oven',
          'toaster','sink','refrigerator','book','clock','vase','scissors','teddy bear','hair drier','toothbrush']


COLORS = np.random.uniform(0, 255, size=(len(LABELS), 3))


In [17]:
pwd

'C:\\Users\\ga383983\\Car-Distance-Estimation-Indication\\yolov2codes'

In [18]:
model = 'C:/Users/ga383983/Car-Distance-Estimation-Indication/yolov2codes/yolov2.weights'
config = 'C:/Users/ga383983/Car-Distance-Estimation-Indication/yolov2codes/yolov2.cfg'
min_confidence=0.14

net = cv2.dnn.readNetFromDarknet(config,model)

In [19]:
import math

# Finction to indicate the nearness of object from the camera lense
def dist_indicator_area(image, startX, startY, box_width, box_height, width, height):

    crop_img = image[startY:startY+box_height, startX:startX+box_width]
    crop_gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(crop_gray,127,255,0)
    contours, hierarchy = cv2.findContours(thresh, 1, 2)

    if len(contours) != 0:
        #Arear of the largest contour(Coutour fo car) witnin bounding box
        c = max(contours, key = cv2.contourArea)
        bb_area = cv2.contourArea(c)
        #print(bb_area)

        #Area of the whole image
        img_area = height*width 
        #print(img_area)

        #Ratio of area of car vs total area of the image
        ratio = bb_area / img_area
        #print(ratio)

        if ratio > 0.3:
            img_label = "Near"
        else:
            img_label = "Far"
    
    return img_label, ratio


#Loading the image of car
image = cv2.imread('bottle_2.jpg')
#print(image.shape)
height, width, ch = image.shape
resize_img = cv2.resize(image,(225,225))



blob = cv2.dnn.blobFromImage(resize_img, 1.0/255.0, (416, 416), True, crop=False)

#blob = cv2.dnn.blobFromImage(resize_img, 0.007843, (300,300), 127.5)

net.setInput(blob)
predictions = net.forward()
probability_index = 5

for i in range(predictions.shape[0]):
    prob_arr=predictions[i][probability_index:]
    class_index = prob_arr.argmax(axis=0)
    confidence = prob_arr[class_index]
    if confidence > min_confidence:
        x_center=predictions[i][0]*width
        y_center=predictions[i][1]*height
        width_box=predictions[i][2]*width
        height_box=predictions[i][3]*height
        
        x1=int(x_center-width_box * 0.5) # Start X coordinate
        y1=int(y_center-height_box * 0.5)# Start Y coordinate
        x2=int(x_center+width_box * 0.5)# End X coordinate
        y2=int(y_center+height_box * 0.5)# End y coordinate
        
        cv2.rectangle(image,(x1,y1),(x2,y2), (0,255,120),2)
             
        if class_index== 39 :# 13 is the index of Bench object in the LABELS list for the predcition as bench

            img_label, ratio = dist_indicator_area(image, x1, y1, int(width_box), int(height_box), width, height)  
            
            cv2.putText(image, img_label, (x1+2, y1+13), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 80, 20), 2)
            
            print(ratio)
        
cv2.imshow("Output", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

0.2426063766269952
